In [ ]:
#hide
from medtop.core import *
from medtop.nlp_helpers import *
from medtop.preprocessing import *
from nbdev.showdoc import *

# MedTop

> Extracting topics from reflective medical writings

This file will become your README and also the index of your documentation.

## Install

`pip install medtop`

## How to use

Download NLTK data to your machine using `python -m nltk.downloader all`

Import documents from a text file containing a list of all documents

In [ ]:
path_to_file_list = 'data/2019.03.11_DevCorpus/file_list.txt'
my_docs, my_docs_pos, my_docs_loc, raw_sentences = import_docs(path_to_file_list, verbose=False)

You can review the imported documents manually by writing the sentences to disk

In [ ]:
outfile_name = 'output/PREFIX_DocumentSentenceList.txt'
sentences_to_disk(raw_sentences, outfile_name)

**Questions on TF-IDF**  
Amy, I have some confusion on the implementation of `create_tfidf`. In the paper, it seems like the the two options for inputs to this function should be either 1) a list of 665 reflective writing files (Global input corpus) or 2) the input file 

In [ ]:
show_doc(create_tfidf)

<h4 id="create_tfidf" class="doc_header"><code>create_tfidf</code><a href="https://github.com/etfrenchvcu/medtop/tree/master/medtop/nlp_helpers.py#L202" class="source_link" style="float:right">[source]</a></h4>

> <code>create_tfidf</code>(**`documents1`**, **`documents2`**=*`'none'`*)

Input: the path to and name of a files containing the list of documents to use for the tf-idf matrix creation.

In [ ]:
parser.add_argument('-i', metavar='inputfile', type=str, help='Path and name of input file list, listing the document names to process. One text file per document in .txt format is required.', required=True)
parser.add_argument('-c', metavar='tfidfcorpus', type=str, help='Path and name of file containing a list of documents to use for creating the TF-IDF matrix. One text file per document in .txt format is required.', required=True)

## Get the TF-IDF matrix
    if args.include_input_in_tfidf:
        tfidf_dense, dictionary = create_tfidf(args.i, args.c)
    else:
        tfidf_dense, dictionary = create_tfidf(args.c)

In [ ]:
tfidf_dense, dictionary = create_tfidf(args.i, args.c)